# CHEM 60 - March 20th, 2024 (Numerical Solutions for ODEs)

This is another integration class, but instead of finding the area under curves, we are using numerical integration to tell us how chemical systems evolve over time. This means we need to learn how to find numerical solutions to systems of ordinary differential equations.

For this material, there are two "in-class" notebooks. This is the one I want you all to finish in our class time together. It covers two of the classic methods for solving ODEs - the Euler method and Runga-Kutta. You'll get practice working with this code today.

Key things to take away from this class: integrating ODEs lets us make predictions for how chemical reactions proceed over time (so long as we know what elementary reactions should occur). This applies from synthetic chemistry all the way to simulations of the atmosphere (the paper Yuki and Max picked in our first class is a nice inspiration for this!)

Another key thing to take away is that our ability to do this numerical integration relies on choosing an appropriate step size. This is quite similar to the sensitivity to step size or learning rate you've seen in other classes already. In this application, we have to choose a time step for our numerical integration. That step needs to be small enough that only tiny changes to our chemical composition happen during it (but if we make it too small, our simulations will never finish).

Ah, the constant trade off between accuracy and computational efficiency (and requires you to have chemical intuition about the system to make good choices).

To get started, click on '**File**' in the left menu, then '**Save a copy in Drive**' to ensure you are editing *your* version of this assignment (if you don't, your changes won't be saved!). After you click '**Save a copy in Drive**' a popup that says **Notebook copy complete** should appear, and it may ask you to <font color='blue'>**Open in a new tab**</font>. When open, your new file will be named `Copy of CHEM60_Class_13_....ipynb` (you may want to rename it before/after you move it to your chosen directory).

#Imports

Here are the Python imports that we will need today. We don't need much to do this! The default formatting stuff is here too.

Run the below code block to get started.

In [ ]:
# Standard library imports
import math as m

# Third party imports
import matplotlib.pyplot as plt
import numpy as np

# This part of the code block is telling matplotlib to make certain font sizes exra, extra large by default
params = {'legend.fontsize': 'xx-large',
         'axes.labelsize': 'xx-large',
         'axes.titlesize':'xx-large',
         'xtick.labelsize':'xx-large',
         'ytick.labelsize':'xx-large'}
# This line updates the default parameters of pyplot (to use our larger fonts)
plt.rcParams.update(params)

Seriously, look how little stuff we need!



---



On Monday, you got to make movies of water molecules. That type of simulation doesn't just make cool animations, or tell us information about the organization of homogenous solutions, it can also be structured to tell us fundamental information about chemical reactions. When we know from observation that some unknown chemistry is occuring, molecular dynamics simulations can help us figure out what elementary reaction steps must be happening and even potentially what the rates of those steps might be.

The chemistry department seminar yesterday actually featured a wonderful example of this (that happens to relate to the ozone chemistry story that we were going to tell today, albeit in a different region of the atmosphere).

The speaker, Kevin Wilson, was primarily presenting on some very cool instrument development work his group is doing to study kinetics in single particles. He shared a bit from a modelling paper his group published that related to some of their instrumental kinetics work.

> Prophet et al., [Iodide oxidation by ozone at the surface of aqueous microdroplets](https://pubs.rsc.org/en/content/articlelanding/2024/sc/d3sc04254e), Chemical Science, 2024.


![Figure from Prophet et al., 2024 -  (A) Snapshot of the MD simulation where an ozone molecule is adsorbed near the air–water interface. (B) The free energy profile for transferring an ozone molecule through a water slab with 0.28 M NaI and 0.84 M NaCl is displayed. The shaded blue region shows the (scaled and shifted) water density profile. ](https://pubs.rsc.org/image/article/2024/sc/d3sc04254e/d3sc04254e-f4_hi-res.gif)


The focus on the paper wasn't the molecular dynamics simulations though - they wanted to build a larger scale model of the chemistry ($10^{20}$ molecules range). To do this, they needed insights from molecular dynamics to figure out what elementary reactions really were occuring and what determines their rates. With this information, you can model systems using chemical kinetics (ie. solving ODEs). You don't have to read this paper, but it is a cool merger of Monday and Wednesdays class this week (because these things quite often go together!).

# How might we simulate chemical kinetics?

There are actually a couple of different ways to go about simulating a chemical reaction kinetically.

Say we have some molecule, $A$ and we know under certain conditions it turns to some other molecule, $B$. We know the rate constant for this reaction, $k$, and now we might want to know how long will it take for all $A$ to become $B$. This is very straight forward. We don't need molecular dynamics at this point (we assume this magic reaction proceeds as $A \rightarrow B$ with a known rate).

We still have a couple of options for how we do this though.

## Stochastic simulation

This is not the main topic of the class today, but is a nice bridge from the molecular scale of Monday to the macro scale that our ODEs will be describing.

To start, we need to establish the general procedure for simulating this system.

**Step 1:**
First, let's define the initial number of A and B molecules. Let $N_a(0) = 500$ and $N_b(0) = 0$. Tiny from a kinetic modellers perspective, but good sized relative to a molecular dynamics lens.

**Step 2:**
Next, we need to set some probability that a reaction will occur. Since this is an unimolecular reaction (something like thermal decomposition), it would capture the odds that a molecule might fall apart or rearrange to take on another form over some given unit of time. Since this is a made-up reaction, we don't need to worry about units. This is like a rate constant, but those are defined for continuous concentrations of reagants, not single molecule probabilities.

**Step 3:** Time Discretization. We'll simulate the system over a period of 100 "seconds" (the time units are also made up here). We need to define a time-step for our simulation. Choose a small enough $dt$ to ensure the system's dynamics are accurately captured. Let's take dt = 0.1

Here is a simple Python code example:


In [ ]:
# Set parameters and initial values
Na, Nb = 500, 0
k = 1
tmax = 100
dt = 0.1

Prepare arrays to store the numbers of $A$ and $B$ molecules

In [ ]:
t = np.arange(0, tmax, dt)
Na_values = np.zeros_like(t)
Nb_values = np.zeros_like(t)

Now loop through our descretized time and randomly allow $A$ to turn to $B$ with a given probability.

In [ ]:
for i in range(len(t)):
    # Store current number of molecules
    Na_values[i] = Na
    Nb_values[i] = Nb
    # Decide whether an A->B reaction occurs
    reaction_occurs = (np.random.random() < k * dt)
    if reaction_occurs:
        # Adjust numbers of molecules due to reaction
        Na -= 1 # lose an A
        Nb += 1 # means gain a B

A lot is happening in the above, but the most important line to understand is:

`reaction_occurs = (np.random.random() < k * dt)`

This is our Python way of probabilistically determining whether a reaction occurs in a given time step.

Let's go through the parts:

1. [`np.random.random()`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.random.html): This creates a (pseudo) random number between 0 and 1, with any given floating point number between [0, 1) equally probable (you probably saw this notation in intro CS - the square bracket means 0 is included and the round bracket means 1 is not included).

2. `k * dt`: This product is a probability created from the reaction rate and the *time step*. The reaction rate 'k' is the probability of the reaction occurring in a unit time period. When we multiply it by 'dt' (a small time step), we get the probability of the reaction occurring in our chosen time increment.

3. `<`: This is a less than comparison (I probably didn't need to write this one out).

4. `reaction_occurs =`: This is assigning the result of the comparison to the variable `reaction_occurs`. This will store a True or False (Boolean) value.

The entire line can be interpreted as, "In this time step, if a random number between 0 and 1 is less than the derived reaction probability (k*dt), then a reaction occurs (reaction_occurs=True), else it doesn't occur (reaction_occurs=False)".

This is one way we could model the inherent randomness of chemical reactions (those times when your path through the PES went backwards!). During a given time step, the reaction may or may not take place.


Let's make a plot to look at it.

In [ ]:
plt.figure(figsize=[8,5])

plt.plot(t,Na_values, '#d95f02', label='$A$', linewidth=5)
plt.plot(t,Nb_values, '#7570b3', label='$B$', linewidth=5)

plt.title('Chemical Kinetics Simulation: A → B')
plt.xlabel('Time (a small unit!)')
plt.ylabel('Number of Molecules')
plt.legend()
plt.grid(True)
plt.show()

Okay...

## PRACTICE QUESTION

With your neighbours, try changing some values about. What happens when we increase the length of time we are running this simulation? Molecule number? Why is this maybe not what we do for most chemical kinetics simulations?



---



***notes***



---



This approach of tracking individual molecules like this is really only suitable when dealing with relatively small numbers of molecules where the stochastic nature of chemical reactions is important. For larger systems, where the numbers of molecules are in the order of Avogadro's number (6.02 x $10^{23}$), a *deterministic* approach (no random numbers needed) based upon concentration changes is usually much more efficient computationally, and just as accurate. When you do have smaller systems, the above can be improved and formalized into something known as the [Gillespie algorithm](https://www.dna.caltech.edu/courses/cs191/paperscs191/gillespie1.pdf) that does work quite well (but we aren't going to go into detail on that one in this course).

What are we actually going to do? Solve ordinary differential equations!




# Intro to solving Ordinary Differential Equations

Let's take the same reaction in the deterministic view (ie. not stochastic, no random probability needed because that information all gets bundled into the rate constant. Deterministic means that if I have the same initial conditions, my simulation should look the same each time I run it). This is the standard approach for simulating chemical reactions when we have a very large number of molecules, and, importantly, known rate constants. This is a very different genre of simulation than what you find in molecular dynamics because here we are starting from the assumption that we already know the chemical mechanism taking place (we need to be able to write the mechanism down to solve these equations). Molecular dynamics and chemical kinetics simulations can work together - one can help us determine what the elementary steps and likely reaction rates are (experiment plays a huge part here too) and then the other can help us simulate enormous systems (like the whole atmosphere!).

This transformation from molecule counts to concentrations makes the problem a continuous one, rather than discrete, for our molecular population. Thus we will formulate the problem as one of solving a system of ordinary differential equations (ODEs). Importantly, **time is still discrete when it comes to numerical solutions.**

## What are ODEs?

You have seen a bit of this in other CORE courses already. An ordinary differential equation (ODE) is an equation involving a function and its derivatives. That sounds extremely generic, but essentially, that's it - being able to write ODEs lets us describe how quantities change with respect to another quantity, quite often time.

Imagine some function $r(t)$ that represents the position of a molecule at time $t$. Its first derivative $\frac{dr}{dt} = r'(t)$ represents the velocity of the molecule. An ODE is an equation that ties those two things together (you saw one on Monday!). ODEs are just a general type of equation - they come up across pretty much all areas of STEM. Being able to numerically solve them for kinetics applications is being able to solve ODEs in pretty much all applications.


### Why "Ordinary"?

It's not an insult - "ordinary" in Ordinary Differential Equations is used to distinguish ODEs from other types of differential equations, mainly partial differential equations (PDEs). Stochastic Differential Equations are another flavour too, but less commonly encountered in undergrad.

In an Ordinary Differential Equation, the function being considered depends on only one independent variable and the derivatives in the equation are regular derivatives with respect to that one independent variable. This is in contrast with Partial Differential Equations where the function depends on two or more independent variables and the equation involves partial derivatives with respect to these variables.

 So, "ordinary" just means that we're only dealing with one independent variable and its regular derivatives in the equation. For example in the equation $dr/dx = x\times r$, we imagine '$r$' depends only on one independent variable '$x$', so this is an ODE. While for something like the heat equation, $∂u/∂t = ∂^2u/∂x^2$, '$u$' depends on two independent variables '$x$' and '$t$', making it a PDE! Solving these is more challenging, but also a major topic in traditional scientific computing courses.

## Back to our chemical example

Let's think about our example of a simple first-order ODE that describes a basic chemical reaction $A \rightarrow B$.

The ODE that represents this reaction is: $\frac{d[A]}{dt} = -k[A]$.

This equation says that the rate of change in the concentration of species $A$ over time is proportional to the current concentration of species $A$, where $k$ is the rate constant of the reaction. This is the rate law for a first-order reaction! Yay, chemistry.

While this above equation is quite simple to solve (it has an exact solution!). Like the examples we did when studying numerical integration, starting with systems with exact solutions makes a lot of sense - we want to know if our numerical method works before we use it more widely.

In general, ODEs are not exactly solvable and must be tackled using numerical methods like **Euler's method** or **Runge-Kutta** methods for an approximate solution. These methods work by calculating a series of small steps, each step moving forward a certain amount in the independent variable (e.g., time) and making an approximation of the function and its derivatives at that point. With these methods, we can gain a pretty good understanding of the behaviour of the system the ODE is describing.

In the case of our chemical reaction, the ODE provides a mathematical model of how the concentrations of $A$ and $B$ evolve over time due to the reaction.

Getting back to our chemical example:

Let's remind our selves that we what to simulate the concentrations of two species, we'll write them as $[A]$ and $[B]$. The reaction we have is $A \rightarrow B$ with rate constant $k$.

That means, remembering our rate laws that:
\begin{align}
\frac{d[A]}{dt} &= -k \times [A] \\
\frac{d[B]}{dt} &= k \times [A]
\end{align}

This is now a *coupled* system of ODEs (ie. we have two ODEs we want to solve together). To solve these numerically, we'll need initial conditions for $A$ and $B$ (ie. the $t=0$ concentrations).


# Euler's method

Because of its simplicity, we will use a numerical method to solve this system of ODEs,  Euler's method. Conceptually, Euler's method starts with an initial point (concentrations of our two compounds), takes a step forward in time based on the derivative at that point (i.e., how fast those concentrations are changing), and then moves on to the next point, repeating the process.

Here is Euler's method formally:

Given the *initial value problem*: $\frac{dr(t)}{dt} = f(t, r(t))$ for $r(t_0) = r_0$, Euler's method generates an approximate solution using steps of equal size, $dt$, to advance from one approximate solution to the next.

The formula used to advance from $r_n$ to $r_{n+1}$ is:

\begin{align}
r_{n+1} = r_n + dt \times f(t_n, r_n)
\end{align}

Here $r_n$ are the approximate solution values at discrete points in the time-domain, $t_n = t_0 + n \times dt$. This formula is derived from the first order Taylor expansion of $r(t)$ about $t_n$ (you saw this on Monday with the integrator!).

For our kinetics example with the ODE $\frac{d[A]}{dt} = -k[A]$ and $[A(t_0)] = [A]_{0}$, Euler's method step would look as follows:

\begin{align}
[A]_{n+1} = [A]_{n} - dt \times k \cdot [A]{n}
\end{align}

This formula is repeated for each step $n$ over the chosen time interval, each time generating an estimate of $[A]$ at a future time point.

Although Euler's method is relatively simple, it's not always the most accurate method for solving ODEs because it uses only first order approximations. Depending on the step size $dt$ and the particular dynamics of the system, the local linearity assumption (ie. that we can take the first-order Taylor expansion approximation and have it be a decent) might not give a good representation of the chemistry. Still, it provides reasonably good approximations to the solution of ODEs and offers a good starting point for understanding numerical integration of ODEs. "Higher order" methods (more complex schemes) share a critical feature it lets us think about - we have to make time discrete and step through time in chunks (and this is sometimes very hard to do).

Here's how you might implement this in Python:

In [ ]:
# Set parameters and initial conditions
k = 0.01
A_conc = 500000
B_conc = 0

# Time step
dt = 0.1
tmax = 100

This next part is where Euler's method is happening:

In [ ]:
# Prepare array to store the concentrations
t = np.arange(0, tmax, dt)
A_values = np.zeros_like(t)
B_values = np.zeros_like(t)

# Euler's method
for i in range(len(t)):
    A_values[i] = A_conc
    B_values[i] = B_conc

    # Compute derivative
    dA = -k * A_conc
    dB  = k * A_conc

    # Update quantities
    A_conc += dA * dt
    B_conc += dB * dt

And a plot:

In [ ]:
# Plotting
plt.figure(figsize=[8,5])
plt.plot(t, A_values, '#d95f02', label='$A$', linewidth=5)
plt.plot(t, B_values, '#7570b3', label='$B$', linewidth=5)
plt.legend()
plt.xlabel('Time (this could be a bigger time unit!)')
plt.ylabel('Concentration')
plt.show()

What did this do?

## PRACTICE QUESTION

Discuss with your partner - what is the above? Why is it different than the stochastic version?



---



**notes**



---



### Let's compare it with the exact solution:

Were these solutions... good? Thankfully, we can check, because this example has an exact solution.


Remembering that we are starting from: $\frac{d[A]}{dt} = -k[A].$

Rewriting the terms and integrating both sides, we get:
\begin{align}
\int \frac{1}{[A]} d[A] = -k \int dt
\end{align}

This leads to:

\begin{align}
ln[A] = -kt + C
\end{align}

Solving this for $[A]$ gives:

\begin{align}
[A(t)]  = e^{-kt + C} = Ce^{-kt}
\end{align}

The constant C can be found from the initial conditions. If at $t=0$, the concentration of A is $[A(t=0)] = [A_{0}]$, then C = $[A_{0}]$.

Our final solution for $[A]$ is thus:

\begin{align}
[A(t)] = [A_{0}] e^{-kt}
\end{align}

This solution indicates that the concentration of $A$ decreases exponentially over time, which is consistent with the characteristics of a first-order reaction (such as our reaction A -> B).

In [ ]:
exact_solution_for_A = A_values[0]*np.exp(-k*t)
# Plotting
plt.figure(figsize=[8,5])
plt.plot(t, A_values, c='#d95f02', label='$A$ Euler', linewidth=5)
plt.plot(t, exact_solution_for_A, c = 'k', linestyle = '--', label = '$A$ exact')
plt.plot(t, B_values, '#7570b3', label='$B$ Euler', linewidth=5)
plt.legend()
plt.xlabel('Time')
plt.ylabel('Concentration')
plt.show()

## PRACTICE QUESTION

Add the exact solution for the change in concentration of $B$ over time. This is just a good chance to practice modifying code.



---



# Runge-Kutta

While Euler's method uses only the information from the starting point of a step to take the next step, Runge-Kutta methods use information perhaps a bit more wisely by combining several intermediate steps to take the actual step. The Fourth Order Runge-Kutta (RK4) method does this by making four 'trial' steps to get the final result. It's like a 'weighted average' of the derivatives. This is somewhat analogous to the different genres of numerical integration we thought about before the break - think rectangle rule vs Simpson's method.

Here's the basic form of RK4:

1. $h_1 = dt \times f(t_n, [A]_n)$
2. $h_2 = dt \times f(t_n + \frac{dt}{2}, [A]_n + \frac{h_1}{2})$
3. $h_3 = dt \times f(t_n + \frac{dt}{2}, [A]_n + \frac{h_2}{2})$
4. $h_4 = dt \times f(t_n + dt, [A]_n + h_3)$
5. $[A]_{n+1} = [A]_n + \frac{1}{6} (h_1 + 2h_2 + 2h_3 + h_4)$

Each $h$ is an estimate of the derivative (slope) at different points within the actual time step. (Note, these "mini" steps are often written as $k_1$, etc. instead of $h_1$, etc., but we reserve $k$ typically for rate constants (which will appear in these same equations), so when you encounter Runge-Kutta outside of chemistry, the above will look a bit different.  

And here's how it looks in Python code:

In [ ]:
A_values_RK = np.zeros_like(t)
B_values_RK = np.zeros_like(t)

# reset the initial conditions (we changed these objects last time)
A_conc = A_values[0]
B_conc = B_values[0]

# Fourth-Order Runge-Kutta method
for i in range(len(t)):
    A_values_RK[i] = A_conc
    B_values_RK[i] = B_conc

    # Compute intermediate RK4 steps for A
    h1_A = dt * (-k * A_conc)
    h2_A = dt * (-k * (A_conc + h1_A * 0.5))
    h3_A = dt * (-k * (A_conc + h2_A * 0.5))
    h4_A = dt * (-k * (A_conc + h3_A))

    # update quantities for A
    A_conc += (h1_A + 2*h2_A + 2*h3_A + h4_A) / 6

    # Compute intermediate RK4 steps for B
    h1_B = dt * (k * A_conc)
    h2_B = dt * (k * (A_conc + h1_A * 0.5))
    h3_B = dt * (k * (A_conc + h2_A * 0.5))
    h4_B = dt * (k * (A_conc + h3_A))

    # update quantities for B
    B_conc += (h1_B + 2*h2_B + 2*h3_B + h4_B) / 6

Let's plot it!

In [ ]:
# Plotting
plt.figure(figsize=[8,5])
plt.plot(t, A_values, c='#d95f02', label='$A$ Euler', linewidth=8)
plt.plot(t, A_values_RK, c='#fdcdac', label='$A$ RK4', linewidth=4)
plt.plot(t, exact_solution_for_A, c = 'k', linestyle = '--', label = 'A exact')

plt.plot(t, B_values, c='#7570b3', label='$A$ Euler', linewidth=8)
plt.plot(t, B_values_RK, c='#cbd5e8', label='$A$ RK4', linewidth=4)

plt.legend()
plt.xlabel('Time')
plt.ylabel('Concentration')
plt.show()

You might think, what is the point? That got the exact same solution as the first-order Euler scheme and had a whole bunch more math (so doesn't that make it less efficient?). For this overly simple system, you are right! In fact, the exact (analytical) solution exists anyways. For more realistic systems of equations, your method will matter a lot and Euler has limitations. Runge-Kutta sadly does too though (we don't have time to go into its more impressive friends, but if you run into a system of ODEs that Runge-Kutta fails for, you probably want *Backward differentiation formula* in your life instead).

# A more complicated example

If we want to think about a chemical compound $X$ that is produced via:
$$\require{mhchem}$$   
\begin{align}
\cee{A + B &-> X} &\quad\quad k_1
\end{align}
and lost via:
\begin{align}
\cee{A + X &-> C} &\quad\quad k_2
\end{align}

>
We can find how it evolves through time by writing the differential equation form of the rate laws,
\begin{align}
\frac{d[X]}{dt}& = +k_1[A][B] - k_2[A][X] \\
\frac{d[A]}{dt}& = -k_1[A][B] - k_2[A][X] \\
\frac{d[B]}{dt}& = -k_1[A][B] \\
\frac{d[C]}{dt}& = +k_2[A][X] \\
\end{align}

Now we have a system of coupled-differential equations, and we can solve them numerically. Let's look at this one now.

In [ ]:
# Set parameters and initial conditions
k1 = 1E-10
k2 = 1E-12
A_conc = 5E11
B_conc = 3E11
X_conc = 5E11

# Time step
dt = 0.02
tmax = 10
t = np.arange(0, tmax, dt)

## PRACTICE QUESTION (in the Google Doc)

Discuss with your partner - why didn't I set the initial concentration of C? Add your comments to the [Google Doc](https://docs.google.com/document/d/1b5hrEgY8NONuxNHluZjLXKUJnCnEoyjXPEwZLXKJPmE/edit?usp=sharing)for Q1.



---



**notes**



---

Let's see how Euler's method does here:

In [ ]:
# Prepare array to store the concentrations
A_values = np.zeros_like(t)
B_values = np.zeros_like(t)
X_values = np.zeros_like(t)

# Euler's method
for i in range(len(t)):
    A_values[i] = A_conc
    B_values[i] = B_conc
    X_values[i] = X_conc

    # Compute derivative
    dA = -k1 * A_conc * B_conc - k2 * A_conc * X_conc
    dB = -k1 * A_conc * B_conc
    dX = k1 * A_conc * B_conc - k2 * A_conc * X_conc

    # Update quantities
    A_conc += dA * dt
    B_conc += dB * dt
    X_conc += dX * dt

And plot:

In [ ]:
# Plotting
plt.figure(figsize=[8,5])
plt.plot(t, A_values, c='#d95f02', label='$A$ Euler', linewidth=5)
plt.plot(t, B_values, c='#7570b3', label='$A$ Euler', linewidth=5)
plt.plot(t, X_values, c='#1b9e77', label='$X$ Euler', linewidth=5)
plt.legend()
plt.xlabel('Time')
plt.ylabel('Concentration')
plt.show()

## PRACTICE QUESTION

Try changing the time step! See what happens.


---



**notes**



---



Let's see what this looks like with Runge Kutta now. Let's create a time step seperate from the one we used with Euler this time.

In [ ]:
# Time step
dt = 0.02
tmax = 10
t_RK = np.arange(0, tmax, dt)

And do the computation now. This is a lot more code. It is, in fact, larger than it needs to be, but it can be helpful to see everything spelled out like this.

In [ ]:
A_values_RK = np.zeros_like(t_RK)
B_values_RK = np.zeros_like(t_RK)
X_values_RK = np.zeros_like(t_RK)

# reset the initial conditions (we changed these objects last time)
A_conc = A_values[0]
B_conc = B_values[0]
X_conc = X_values[0]

# Fourth-Order Runge-Kutta method
for i in range(len(t_RK)):
    A_values_RK[i] = A_conc
    B_values_RK[i] = B_conc
    X_values_RK[i] = X_conc

    ####### First steps first (occurs at initial position)######################
    # Compute intermediate RK4 steps for A
    h1_A = dt * (-k1 * A_conc * B_conc - k2 * A_conc * X_conc)
    # Compute intermediate RK4 steps for B
    h1_B = dt * (-k1 * A_conc * B_conc)
    # Compute intermediate RK4 steps for X
    h1_X = dt * (k1 * A_conc * B_conc - k2 * A_conc * X_conc)

    ###### Second step next (occurs at mid-point)###############################
    h2_A = dt * (-k1 * (A_conc + h1_A * 0.5) * (B_conc + h1_B * 0.5) - k2 * (A_conc + h1_A * 0.5) * (X_conc + h1_X * 0.5))
    h2_B = dt * (-k1 * (A_conc + h1_A * 0.5) * (B_conc + h1_B * 0.5))
    h2_X = dt * (+k1 * (A_conc + h1_A * 0.5) * (B_conc + h1_B * 0.5) - k2 * (A_conc + h1_A * 0.5) * (X_conc + h1_X * 0.5))

    ###### Third step next (occurs at mid-point) ###############################
    h3_A = dt * (-k1 * (A_conc + h2_A * 0.5) * (B_conc + h2_B * 0.5) - k2 * (A_conc + h2_A * 0.5) * (X_conc + h2_X * 0.5))
    h3_B = dt * (-k1 * (A_conc + h2_A * 0.5) * (B_conc + h2_B * 0.5))
    h3_X = dt * (+k1 * (A_conc + h2_A * 0.5) * (B_conc + h2_B * 0.5) - k2 * (A_conc + h2_A * 0.5) * (X_conc + h2_X * 0.5))

    ##### Fourth step next (occurs at end point) ###############################
    h4_A = dt * (-k1 * (A_conc + h3_A) * (B_conc + h3_B) - k2 * (A_conc + h3_A) *  (X_conc + h3_X))
    h4_B = dt * (-k1 * (A_conc + h3_A) * (B_conc + h3_B))
    h4_X = dt * (+k1 * (A_conc + h3_A) * (B_conc + h3_B) - k2 * (A_conc + h3_A) *  (X_conc + h3_X))

    # update quantities for A
    A_conc += (h1_A + 2*h2_A + 2*h3_A + h4_A) / 6
    # update quantities for B
    B_conc += (h1_B + 2*h2_B + 2*h3_B + h4_B) / 6
    # update quantities for X
    X_conc += (h1_X + 2*h2_X + 2*h3_X + h4_X) / 6

And the plot:

In [ ]:
# Plotting
plt.figure(figsize=[8,5])
plt.plot(t, A_values, c='#d95f02', label='$A$ Euler', linewidth=8)
plt.plot(t_RK, A_values_RK, c='#fdcdac', label='$A$ RK4', linewidth=4)
plt.plot(t, B_values, c='#7570b3', label='$B$ Euler', linewidth=8)
plt.plot(t_RK, B_values_RK, c='#cbd5e8', label='$B$ RK4', linewidth=4)
plt.plot(t, X_values, c='#1b9e77', label='$X$ Euler', linewidth=8)
plt.plot(t_RK, X_values_RK, c='#b3e2cd', label='$X$ RK4', linewidth=4)
plt.legend()
plt.xlabel('Time')
plt.ylabel('Concentration')
plt.show()

## PRACTICE QUESTION

If you kept `dt = 0.02` for both solvers, what happens? Discuss with your partner. Do Euler and Runge-Kutta always agree?

Try out different time steps for each. How do you know which one is 'right'? What if you try different initial concentrations?


---



**notes**



---

## PRACTICE QUESTION (in the [Google Doc](https://docs.google.com/document/d/1b5hrEgY8NONuxNHluZjLXKUJnCnEoyjXPEwZLXKJPmE/edit?usp=sharing))

For Q2, share a comparison figure of Euler vs RK4. Make clear what timesteps you are using for each of them (they do not have to use the same time step as each other). This figure can show them both working well, one working poorly, or both working poorly. What are potential reasons you might have to choose one method or the other? What if we care about the steady-state concentration of the system? What if we care about the early evolution of the reaction? Can you think of the factors that determine how small the time step should be?



---

When picking a numerical method, there is always a trade off between accuracy and computation time/resources required. For most systems of ODEs, Runge-Kutta's accuracy improvements over the Euler method (and RK's ability to potentially take larger time steps) is worth the greater complexity of the code and increased number of calculations per step (the number of steps is usually what determines how long something takes to run).

The above Runge Kutta code can be made faster too. Each time we repeat the exact same calculation, we are wasting resources. ie. Every  `(A_conc + h1_A * 0.5)` in the below block of code (for example):

>     h2_A = dt * (-k1 * (A_conc + h1_A * 0.5) * (B_conc + h1_B * 0.5) - k2 * (A_conc + h1_A * 0.5) * (X_conc + h1_X * 0.5))
    h2_B = dt * (-k1 * (A_conc + h1_A * 0.5) * (B_conc + h1_B * 0.5))
    h2_X = dt * (+k1 * (A_conc + h1_A * 0.5) * (B_conc + h1_B * 0.5) - k2 * (A_conc + h1_A * 0.5) * (X_conc + h1_X * 0.5))

 takes a tiny bit of time and energy to run. What if we made a new object, say `A_conc_h1_A = (A_conc + h1_A * 0.5)` and just called it every time we needed it instead of re-calculating it 5 times. This might seem like a silly savings, but in a laaaarge model, for a big system of equations, these little savings add up significantly.





# PRACTICE QUESTION

 Your final piece of the in-class work is turning the above Runge Kutta example into a function that drops all of these inefficiencies. I encourage talking this through with your neighbours.

 You want a function that does not repeat calculations. It can take initial concentrations as arguments and rate constants as well as time step and length of simulation time.

 ie.

 def `Runge_Kutta_for_ABX(A_init, B_init, X_init, k1, k2, dt, t_max)`:

it should return `t`, `A_values`, `B_values`, and `X_values`. Test it by calling it and recreating a plot of its output.


---




In [ ]:
# code!



---

Okay. What was this all [for](https://drive.google.com/file/d/1YSXKAqCE84XtYABq4O9r6wxBd_3yrTzR/view?usp=sharing)?



---



# Submit your notebook

It's time to download your notebook and submit it on Canvas. Go to the File menu and click **Download** -> **Download .ipynb**

Then, go to **Canvas** and **submit your assignment** on the assignment page. Once it is submitted, swing over to the homework now and start working through the paper.